### Linear regression using PyTorch built-ins
The last model and training process were implemented using basic matrix operations. But since this is such a common pattern, PyTorch has several built-in functions and classes to make it easy to create and train models.

Let's begin by importing the torch.nn package from PyTorch, which contains utility classes for building neural networks.

In [1]:
import torch.nn as nn
import numpy as np
import torch

As before, we represent the inputs and targets and matrices.

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58],
                  [102, 43, 37], [69, 96, 70], [73, 67, 43],
                  [91, 88, 64], [87, 134, 58], [102, 43, 37],
                  [69, 96, 70], [73, 67, 43], [91, 88, 64],
                  [87, 134, 58], [102, 43, 37], [69, 96, 70]], dtype = 'float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133],
                   [22, 37], [109, 119], [56, 70], 
                   [81, 101], [119, 133], [22, 37],
                   [109, 119], [56, 70], [81, 101],
                   [119, 133], [22, 37], [109, 119]], dtype = 'float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

We are using 15 training examples thus time, to illustrate how to work with large datasets in small batches.

### Datasets and DataLoader
We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [3]:
from torch.utils.data import TensorDataset

In [4]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [5]:
from torch.utils.data import DataLoader

In [6]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle = True)

The data loader is typically used in a for-in loop. Let's look at an example.

In [7]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[102.,  43.,  37.],
        [ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [ 69.,  96.,  70.],
        [ 91.,  88.,  64.]])
tensor([[ 22.,  37.],
        [119., 133.],
        [ 56.,  70.],
        [109., 119.],
        [ 81., 101.]])


In each iteration, the data loader returns one batch of data, with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

### nn.linear
Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automaically.

In [8]:
# Define the model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.5213,  0.0569, -0.1521],
        [-0.3570,  0.4834, -0.5513]], requires_grad=True)
Parameter containing:
tensor([-0.3789,  0.2236], requires_grad=True)


PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in thr model. For our linear regression model, we have one weight matrix and one bias matrix.

In [9]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.5213,  0.0569, -0.1521],
         [-0.3570,  0.4834, -0.5513]], requires_grad=True),
 Parameter containing:
 tensor([-0.3789,  0.2236], requires_grad=True)]

We can use the model to generate predictions in the exact same way as before:

In [10]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 34.9523, -17.1507],
        [ 42.3377, -25.0014],
        [ 43.7832,   1.9726],
        [ 49.6175, -35.7974],
        [ 30.4111, -16.5885],
        [ 34.9523, -17.1507],
        [ 42.3377, -25.0014],
        [ 43.7832,   1.9726],
        [ 49.6175, -35.7974],
        [ 30.4111, -16.5885],
        [ 34.9523, -17.1507],
        [ 42.3377, -25.0014],
        [ 43.7832,   1.9726],
        [ 49.6175, -35.7974],
        [ 30.4111, -16.5885]], grad_fn=<AddmmBackward>)

### Loss function 
Instead of defining a loss function manually, we can use the built-in loss function mse_loss.

In [11]:
# Import the nn.functional
import torch.nn.functional as F

The nn.functional package contains many useful loss functions and several other utilities.

In [12]:
# Define loss function
loss_fn = F.mse_loss


Let's compute the loss for the current predictions of our model.

Za definiciju neke funkcije ?funkcija, npr. ?nn.Linear

In [13]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(7885.7769, grad_fn=<MseLossBackward>)


### Optimizer
Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer optim.SGD . SGD stands for stohastic gradient descent. It is called stohastic bacause samples are selected in batches (often with random shuffling) instead of as a single group.

In [14]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)   # lr-learning rate

Note that model.paremeters() is passed as an argument to optim.SGD, so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate which controls the amount by which the parameters are modified.

### Train the model
We are now ready to train the model. We'll follow the exact same process to implement gradient descent:
1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero
The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. let's define a utility function fit which trains the model for a given number of epochs.

In [15]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train batches of data
        for xb, yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update paameters using gradient descent
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
            
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

 Some things to note above:
 - We use the data loader defined earlier to get batches of data for every iteration.
 - Instead of updating parameters (weights and biases) manually, we use opt.step to perform the update, and opt.zero_grad to reset the gradients to zero.
 - We've also added a log statement which prints the loss from the last batch of data for every 10th epoch, to track the progress of training. loss.item returns the actual value stored in the loss tensor.
 
 
Let's train the modsel for 100 epochs.

In [19]:
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 59.9954
Epoch [20/100], Loss: 63.7031
Epoch [30/100], Loss: 52.0670
Epoch [40/100], Loss: 70.7784
Epoch [50/100], Loss: 38.1551
Epoch [60/100], Loss: 57.9909
Epoch [70/100], Loss: 57.7340
Epoch [80/100], Loss: 38.8623
Epoch [90/100], Loss: 38.1174
Epoch [100/100], Loss: 19.1641


Let's generate predictions using our model and verify that they're close to our targets.

In [20]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.1411,  70.7606],
        [ 81.1948,  95.7814],
        [124.4094, 142.1648],
        [ 23.0004,  39.8352],
        [ 98.6036, 109.1187],
        [ 58.1411,  70.7606],
        [ 81.1948,  95.7814],
        [124.4094, 142.1648],
        [ 23.0004,  39.8352],
        [ 98.6036, 109.1187],
        [ 58.1411,  70.7606],
        [ 81.1948,  95.7814],
        [124.4094, 142.1648],
        [ 23.0004,  39.8352],
        [ 98.6036, 109.1187]], grad_fn=<AddmmBackward>)

In [21]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [109., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [109., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [109., 119.]])